In [132]:
#User Items
#Importo Librerias
import pandas as pd
import gzip

PlayTimeGenre

In [133]:
# Asigno al DataFrame el archivo csv df_sg.csv
#df_sg=pd.read_csv('/content/drive/MyDrive/PI_ML_OPS-PT/Dataset/df_sg.csv')
df_sg=pd.read_csv('./Dataset/df_sg.csv')

In [118]:
# Asigno al DataFrame el archivo csv df_ui.csv.gz
#df_ui = pd.read_csv('/content/drive/MyDrive/PI_ML_OPS-PT/Dataset/df_ui.csv.gz', compression='gzip')
df_ui = pd.read_csv('./Dataset/df_ui.csv.gz', compression='gzip')

In [119]:
# Crear dataframe base de PlayTimeGenre
df_sgui_pg=pd.merge(df_sg[['item_id','genres','release_date']],df_ui[['item_id','playtime_forever']],on='item_id',how='left')
df_sgui_pg.head()

,item_id,genres,release_date,playtime_forever
0,761140,Action,2018,NaN
1,761140,Casual,2018,NaN
2,761140,Indie,2018,NaN
3,761140,Simulation,2018,NaN
4,761140,Strategy,2018,NaN


In [120]:
df_sg_r3 =df_sg[df_sg['item_id']==1250]
df_sg_r3
df_sg_r4 =df_sgui_pg[df_sgui_pg['item_id']==1250]
df_sg_r4

,item_id,genres,release_date,playtime_forever
6683216,1250,Action,2009,10006.0
6683217,1250,Action,2009,428.0
6683218,1250,Action,2009,1646.0
6683219,1250,Action,2009,3546.0
6683220,1250,Action,2009,16435.0
...,...,...,...,...
6696497,1250,Action,2009,1.0
6696498,1250,Action,2009,502.0
6696499,1250,Action,2009,2910.0
6696500,1250,Action,2009,268.0


In [5]:
# Elimino nulos
df_sgui_pg.dropna(inplace=True)

In [6]:
# Creo dataframe final para PlayTimeGenre
df_PlayTimeGenre = df_sgui_pg.groupby(['genres', 'release_date'])['playtime_forever'].sum().reset_index()
df_PlayTimeGenre.head()

,genres,release_date,playtime_forever
0,Action,1983,3582.0
1,Action,1984,384.0
2,Action,1988,16243.0
3,Action,1989,607.0
4,Action,1990,18787.0


In [7]:
# Guardo dataframe en PlayTimeGenre.csv
#df_PlayTimeGenre.to_csv('/content/drive/MyDrive/PI_ML_OPS-PT/Dataset/df_PlayTimeGenre.csv',index=False)
df_PlayTimeGenre.to_csv('./Dataset/df_PlayTimeGenre.csv',index=False)

In [8]:
#Funcion PlayTimeGenre
genre='RPG'
#df_playtimegenre= pd.read_csv('/content/drive/MyDrive/PI_ML_OPS-PT/Dataset/df_PlayTimeGenre.csv')
df_playtimegenre= pd.read_csv('./Dataset/df_PlayTimeGenre.csv')
df_filtered_by_genre = df_playtimegenre[df_playtimegenre['genres'] == genre]
gender_hours = df_filtered_by_genre.groupby('release_date')['playtime_forever'].sum()
max_time_play_gender = gender_hours.idxmax()
result = {
  "Año de lanzamiento con más horas jugadas para Género {}".format(genre) : max_time_play_gender
  }
result

{'Año de lanzamiento con más horas jugadas para Género RPG': 2011}

UserForGenre

In [14]:
# Creo Lista de Generos
df_generos_diferentes = df_sg['genres'].unique()
df_generos_diferentes.sort()

In [77]:
# Relaciono dataframes steam_games con user_items
df_sgui_gu=pd.merge(df_sg[['item_id','genres','release_date']],df_ui[['item_id','playtime_forever','user_id']],on='item_id',how='left')
df_sgui_gu.dropna(inplace=True)

# Creo lista de generos
usuario_por_genero = []
df_UserForGenre = pd.DataFrame(columns=['user_id', 'genres', 'release_date', 'playtime_forever'])


for genero in df_generos_diferentes:
    # Filtramos las filas del género actual
    df_genero = df_sgui_gu[df_sgui_gu['genres'] == genero]
    # Filtramos el usuario del género actual
    if not df_genero.empty:
        # Filtramos al usuario con mayor 'playtime_forever' del género actual
        df_usuario = df_genero.groupby('user_id')['playtime_forever'].sum().div(60).idxmax()
        # Agrupamos los datos por usuario, genero y año y calculamos la suma de 'playtime_forever'
        df_usuario_genero_año = df_genero[df_genero['user_id'] == df_usuario].groupby(['user_id','genres', 'release_date'])['playtime_forever'].sum().div(60).reset_index()
        # Agregamos el df_usuario_genero_año a la lista usuario_por_genero
        usuario_por_genero.append(df_usuario_genero_año)
    else:
        pass
# Convertimos los resultados en un DataFrame
for df in usuario_por_genero:
    # Agregamos al DataFrame df_UserForGenre los dataframes de cada genero y usuario
    df_UserForGenre = pd.concat([df_UserForGenre, df], ignore_index=True)
df_UserForGenre['playtime_forever'] = df_UserForGenre['playtime_forever'].astype('int')
df_UserForGenre.drop(df_UserForGenre[df_UserForGenre['playtime_forever'] == 0].index, inplace=True)
df_UserForGenre = df_UserForGenre.reset_index(drop=True)
df_UserForGenre.head()

,user_id,genres,release_date,playtime_forever
0,Sp3ctre,Action,1995,3
1,Sp3ctre,Action,2000,1177
2,Sp3ctre,Action,2002,3
3,Sp3ctre,Action,2003,127
4,Sp3ctre,Action,2004,2123


In [47]:
# Guardo Dataframe en df_UserForGenre.csv
# df_UserForGenre.to_csv('/content/drive/MyDrive/PI_ML_OPS-PT/Dataset/df_UserForGenre.csv',index=False)
df_UserForGenre.to_csv('./Dataset/df_UserForGenre.csv',index=False)

In [48]:
#Funcion UserForGenre
genre='Action'
# df_userforgenre= pd.read_csv('/content/drive/MyDrive/PI_ML_OPS-PT/Dataset/df_UserForGenre.csv')
df_userforgenre= pd.read_csv('./Dataset/df_UserForGenre.csv')
df_filtered_by_genre = df_userforgenre[df_userforgenre['genres']==genre]

user_hours = df_filtered_by_genre.groupby('user_id')['playtime_forever'].sum()
max_time_play_user = user_hours.idxmax()

max_user_df = df_filtered_by_genre[df_filtered_by_genre['user_id'] == max_time_play_user]
hours_by_year = max_user_df.groupby('release_date')['playtime_forever'].sum().reset_index()

hours_years_list = [
    {'Año':year, 'Horas': hours} for year, hours in zip(hours_by_year['release_date'],hours_by_year['playtime_forever'])
                    ]
result = {
  'Usuario con más horas jugadas para genero {}'.format(genre):max_time_play_user,
  'Horas jugadas': hours_years_list
  }
result

{'Usuario con más horas jugadas para genero Action': 'Sp3ctre',
 'Horas jugadas': [{'Año': 1995, 'Horas': 3},
  {'Año': 2000, 'Horas': 1177},
  {'Año': 2002, 'Horas': 3},
  {'Año': 2003, 'Horas': 127},
  {'Año': 2004, 'Horas': 2123},
  {'Año': 2005, 'Horas': 355},
  {'Año': 2006, 'Horas': 14},
  {'Año': 2007, 'Horas': 1879},
  {'Año': 2008, 'Horas': 3},
  {'Año': 2009, 'Horas': 1805},
  {'Año': 2010, 'Horas': 1301},
  {'Año': 2011, 'Horas': 2581},
  {'Año': 2012, 'Horas': 6304},
  {'Año': 2013, 'Horas': 2007},
  {'Año': 2014, 'Horas': 2178},
  {'Año': 2015, 'Horas': 5125},
  {'Año': 2016, 'Horas': 492},
  {'Año': 2017, 'Horas': 722}]}

UsersRecommend

In [153]:
# Asigno al DataFrame el archivo csv df_ur.csv
#df_ur=pd.read_csv('/content/drive/MyDrive/PI_ML_OPS-PT/Dataset/df_ur.csv')
df_ur=pd.read_csv('./Dataset/df_ur.csv')
# Filtro el dataframe por recomendado positiva o neutral 
df_ur_r = df_ur[
    (df_ur['recommend'] == True) &
    (df_ur['sentiment_analysis'] != 0)] 

df_sg_r = df_sg[['item_id', 'app_name','release_date']].drop_duplicates().reset_index(drop=True)
#Merge de los dataframes de steam_games filtrados y user_reviews filtrados
df_sgur_r=pd.merge(df_sg_r[['item_id', 'app_name','release_date']],df_ur[['item_id','recommend','sentiment_analysis']],on='item_id',how='inner')

In [152]:
# Creo Lista de Años
df_años_diferentes = df_sg['release_date'].unique()
df_años_diferentes.sort()
#df_años_diferentes

# Creo lista de años
app_por_año = []
df_UsersRecommend = pd.DataFrame(columns=['app_name', 'Count', 'release_date'])

for año in df_años_diferentes:
    # Filtramos las filas del género actual
    df_año = df_sgur_r[df_sgur_r['release_date'] == año]
    # Filtramos el usuario del género actual
    if not df_año.empty:
        top_games_año = df_año['app_name'].value_counts().head(3)
        
        # Create a DataFrame from the series
        top_games_año_df = top_games_año.reset_index()
        # Rename the columns
        top_games_año_df.columns = ['app_name', 'Count']
        # Add a new column with the value of the year
        top_games_año_df['release_date'] = df_año['release_date'].iloc[0]  # Assuming the year is the same for all rows

        app_por_año.append(top_games_año_df)
    else:
        pass
# Convertimos los resultados en un DataFrame
for df in app_por_año:
    # Agregamos al DataFrame df_UserForGenre los dataframes de cada genero y usuario
    df_UsersRecommend = pd.concat([df_UsersRecommend, df], ignore_index=True)

df_UsersRecommend.head(60)

,app_name,Count,release_date
0,Sword of the Samurai,1,1989
1,Commander Keen,4,1990
2,LOOM™,1,1990
3,Crystal Caves,1,1991
4,Putt-Putt® Joins the Parade,1,1992
5,Word Rescue,1,1992
6,Cosmo's Cosmic Adventure,1,1992
7,X-COM: UFO Defense,3,1993
8,Putt-Putt® Goes to the Moon,1,1993
9,Litil Divil,1,1993
